<a href="https://colab.research.google.com/github/cfreeman22/CreditCardChurnPrediction/blob/main/CreditCardUsersChurnPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit card churn Prediction
Which customer is most likely to get a loan?
Presented By Christian Freeman  March 2022

## Project Planning
## PLAN ==> Acquire ==> Prepare ==> Explore ==> Model &Evaluate ==> Deliver
Understanding consumer behavior can quite chanlenging, especially when the product we offer our customers is a service rather than a tangible physical products. In the case of predicting which customer is most likely to get a loan,, I will perform the following:

Prepare the data for visualization using a customized module wrangle.py
Identify patterns and insights
Document key observations and relationships between variables
Goals and summary
My goal in this project is to:

Identify the characteristics of customers who are more likely to get a loan, and the potential reasons for this behavior.
Build a ML model to predict which customer may sign up for a loan So that recommendations are made based on the findings to help increase the chances of giving a loan the the right customer.
I will be using Python, Pandas, Matplotlib, Seaborn, and Scikit-Learn libraries to analyse and create ML classification model.
## Initial Questions
What are unique characteristics of our customers?
What factors increase their chances of getting a loan ?
can we predict loan subscribers ?
## Executive Summary
Findings Include: